## TF-IDF
TF-IDF, which stands for Term Frequency-Inverse Document Frequency, is a method for featurizing text which takes into account both word frequency within a single document, as well as the frequency of that word across all documents in a corpus. The purpose is to weight infrequent 'specialty' words more heavily than common words, since those specialty words tend to give us more information about the document.

Words that appear in only a few documents give us a lot of information about the type of document it came from. 
#### This is a typical text featurization pipeline:
```
Tokenization ----> Stop Word Removal ----> Stemming/Lemmatization ----> Bag of words/TFIDF   
```
#### Tokenization
As with preparing text for Naive Bayes, our pipeline starts by tokenizing the corpus into a list of list of strings. At this point, we should also make decisions about capitalization, punctuation, and whether to account for sentence and paragraph boundaries.

#### Stop Words
An ongoing theme in machine learning is that features that do not give a lot of information should be excluded or penalized. In NLP, the first step in reducing our feature-space is to remove stop words--words that appear frequently across many documents and across many topics. They're words that don't help us narrow down a document's unique qualities.

#### Stemming and lemmatization

- Stemming is a more crude technique that follows a set of rules for removing things like plurals, verb endings, and adjectivizations in order to condense many related words down to a single token. It will only ever remove or modify the ends of words, and a lot of the tokens it outputs are not actual words. Here is an example of a word passing through through the Porter Stemmer algorithm.
```
realizations --> realization --> realizate --> realiz
```
- Lemmatizers, on the other hand, have been trained to examine the morphology of each word, to determine its function in the sentence and modify it appropriately. A lemmatizer will generally output a larger feature set than a stemmer. In this case, lemmatizing 'realizations' yields 'realization'--not much of a change when compared to 'realiz'!

### TF-IDF pseudo-code
1. count the total number of documents in the corpus
2. create a vocabulary
3. create a matrix of zeroes of size num_docs * num_words_in_vocab
4. for each word in the vocabulary:  
tally number of documents in which the word appears   
compute inverse document frequency   
store this value   
5. for each document in the corpus:   
for each word in the document's bag of words:    
tally raw term frequency   
multiply term frequency by corresponding inverse document frequency   
store this value at the appropriate location in the matrix   
6. return the filled-in matrix



### IDF

$$idf=log⁡(\frac{N}{n_t+1})$$

### Cosine Simiarity
$$s(a,b) = \frac{a\cdot b}{||a|| ||b||} = \frac{\sum_{i=1}^N a_i b_i}{\sqrt{\sum_{i=1}^N a_i^2 }\sqrt{\sum_{i=1}^N a_i^2}} $$

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import nltk.data
import numpy as np


# There's a function for each section of the sprint as well as some additional
# helper functions.


def main():
    # GET THE DATA
    # set categories = None to get all the data. Will be slow.
    categories = ['comp.graphics', 'rec.sport.baseball', 'sci.med', \
                  'talk.politics.misc']
    newsgroups = fetch_20newsgroups(subset='train', categories=categories)

    # DO TFIDF TRANSFORMATION
    vectorizer = TfidfVectorizer(stop_words='english')
    vectors = vectorizer.fit_transform(newsgroups.data).toarray()

    # FEATURE IMPORTANCES
    top_n(vectorizer, vectors, newsgroups.data, 10)

    # RANKING
    ranking(vectorizer, vectors, newsgroups.filenames,
            get_queries('queries.txt'), 3)

    # SUMMARIZATION
    article = newsgroups.data[1599]  # can choose any article
    summarization(article, categories, 3)


def top_n(vectorizer, vectors, data, n):
    '''
    Print out the top 10 words by three different sorting mechanisms:
        * average tf-idf score
        * total tf-idf score
        * highest TF score across corpus
    '''
    words = vectorizer.get_feature_names()

    # Top 10 words by average tfidf
    # Take the average of each column, denoted by axis=0
    avg = np.sum(vectors, axis=0) / np.sum(vectors > 0, axis=0)
    print "top %d by average tf-idf" % n
    print get_top_values(avg, n, words)
    print

    # Top 10 words by total tfidf
    total = np.sum(vectors, axis=0)
    print "top %d by total tf-idf" % n
    print get_top_values(total, n, words)
    print

    # Top 10 words by TF
    vectorizer2 = TfidfVectorizer(use_idf=False)
    # make documents into one giant document for this purpose
    vectors2 = vectorizer2.fit_transform([" ".join(data)]).toarray()
    print "top %d by tf across all corpus" % n
    print get_top_values(vectors2[0], n, words)
    print


def get_queries(filename):
    '''
    Return a list of strings of the queries in the file.
    '''
    queries = []
    with open('queries.txt') as f:
        for line in f:
            # horrible stuff to get out the query
            queries.append(line.split("   ")[1].split("20")[0].strip())
    return queries


def ranking(vectorizer, vectors, titles, queries, n):
    '''
    Print out the top n documents for each of the queries.
    '''
    tokenized_queries = vectorizer.transform(queries)
    cosine_similarities = linear_kernel(tokenized_queries, vectors)
    for i, query in enumerate(queries):
        print query
        print get_top_values(cosine_similarities[i], 3, titles)
        print


def summarize(article, sent_detector, n):
    '''
    Choose top n the sentences based on max tf-idf score.
    '''
    sentences = sent_detector.tokenize(article.strip())
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform(sentences).toarray()
    # We are summing on axis=1 (total per row)
    total = np.sum(vectors, 1)
    lengths = np.array([len(sent) for sent in sentences])
    return get_top_values(total / lengths.astype(float), n, sentences)


def summarization(article, categories, n):
    '''
    Print top n sentences from the article.
    Print top n sentences from each category.
    '''
    sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

    # summarize an article
    print summarize(article, sent_detector, n)
    print

    for cat in categories:
        newsgroup = fetch_20newsgroups(subset='train', categories=[cat])
        print cat
        # combine all articles into one string to summarize.
        print summarize("\n".join(newsgroup.data), sent_detector, n)


def get_top_values(lst, n, labels):
    '''
    INPUT: LIST, INTEGER, LIST
    OUTPUT: LIST

    Given a list of values, find the indices with the highest n values. Return
    the labels for each of these indices.

    e.g.
    lst = [7, 3, 2, 4, 1]
    n = 2
    labels = ["cat", "dog", "mouse", "pig", "rabbit"]
    output: ["cat", "pig"]
    '''
    return [labels[i] for i in np.argsort(lst)[-1:-n-1:-1]]


if __name__ == '__main__':
    main()
